# **MOUNTING GOOGLE DRIVE**




In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# **LOADING LIBRARIES**

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

# **CONFIGURING PATH**

In [ ]:
path =  "/content/gdrive/MyDrive/IBM_GRM/chest_xray/"
train_path = os.path.join(path,"train")
test_path=os.path.join(path,"val")
val_path=os.path.join(path,"test")

# **NUMBER OF IMAGES IN: TRAIN || TEST || VAL**

In [ ]:
print("Train : "+str(len(os.listdir(train_path+'/NORMAL')) + len(os.listdir(train_path+'/PNEUMONIA')) + len(os.listdir(train_path+'/Covid'))+ len(os.listdir(train_path+'/TB'))))
print("Test  : "+str(len(os.listdir(val_path+'/PNEUMONIA')) + len(os.listdir(val_path+'/NORMAL')) + len(os.listdir(val_path+'/Covid'))+ len(os.listdir(val_path+'/TB'))))
print("Val   : "+str(len(os.listdir(test_path+'/NORMAL')) + len(os.listdir(test_path+'/PNEUMONIA')) + len(os.listdir(test_path+'/Covid'))+ len(os.listdir(test_path+'/TB'))))

Train : 5504
Test  : 704
Val   : 480


# **USING DATA AUGMENTATION**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_Datagen = ImageDataGenerator(
    rescale =1/255,
#     shear_range=10,
    zoom_range = 0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
#     rotation_range=20,
    fill_mode = 'nearest',
)
val_datagen = ImageDataGenerator(
    rescale =1/255,
)

# **TRANSFER LEARNING || BUILDING THE MODEL ARCHITECTURE**

In [ ]:
import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import Xception
base_model = Xception(include_top=False, weights='imagenet')
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation = 'relu',kernel_regularizer= keras.regularizers.l2(l2=0.05)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu',kernel_regularizer=keras.regularizers.l2(l2=0.05)))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.summary()

83697664/83683744 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________

# **CREATING TRAIN || VALIDATION || TEST GENERATORS**

In [ ]:
train_generator=train_Datagen.flow_from_directory(
    train_path,
    target_size=(150,150),
    batch_size= 32,
    
)
validation_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=(150,150),
        batch_size=32,
        
)
test_generator = val_datagen.flow_from_directory(
    test_path,
    target_size=(150,150),
    batch_size=32,
    
)

Found 5504 images belonging to 4 classes.
Found 704 images belonging to 4 classes.
Found 480 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'Covid': 0, 'NORMAL': 1, 'PNEUMONIA': 2, 'TB': 3}

# **USING EXPONENTIAL DECAY**

In [ ]:
from tensorflow.keras.optimizers import Adam
initial_learning_rate = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=650,
    decay_rate=0.9
    #staircase=True
    )

model.compile(optimizer= Adam(lr_schedule), loss='categorical_crossentropy', metrics=["accuracy"])

# **TRAINING THE MODEL**

In [ ]:
# history_new = model.fit(train_generator,
#                               epochs=20,
#                               steps_per_epoch = 5024 // 32,
#                               validation_data = validation_generator,
#                               validation_steps = 640 // 32, verbose=1)

# **EVALUATING THE MODEL**

In [ ]:
model.load_weights('/content/gdrive/MyDrive/IBM_GRM/chest_xray/model_weights/FINALMODELDISEASECLASSIFICATION.h5')

In [ ]:
op = model.evaluate_generator(test_generator)
op

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2006: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[1.084539532661438, 0.9750000238418579]

In [ ]:
print("Accuracy obtained on the test dataset is: {}".format(op[1]))

Accuracy obtained on the test dataset is: 0.9750000238418579
